# Ruff Python code quality formatter

We are using a GPT-4.1-mini, Gemini-2.5-Pro, and Gemma3:270m to modify Python code to a Ruff code quality compliant Python code, eliminating code smells.


In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess
from IPython.display import Markdown, display


In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

In [3]:
# Connect to client libraries

openai = OpenAI()

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
ollama_url = "http://localhost:11434/v1"

gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [4]:
models = ["gpt-4.1-mini", "gemini-2.5-pro", "gemma3:270m"]

clients = {"gpt-4.1-mini": openai, "gemini-2.5-pro": gemini, "gemma3:270m": ollama}

In [5]:
system_prompt = f"""
Your task is to modify Python code to a Ruff code quality compliant Python code, eliminating code smells.
The code needs to produce an identical output as the original version.
"""

def user_prompt_for(code):
    return f"""
Make this python code Ruff code quality compliant.
Python code to modify:

```python
{code}
```
"""

In [6]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [12]:
def implement_ruff_code_quality(model, code):
    client = clients[model]
    response = client.chat.completions.create(model=model, messages=messages_for(code))
    reply = response.choices[0].message.content
    reply = reply.replace('```python','').replace('```','')
    return reply

In [8]:
def run_code(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [9]:
python_code = """# Be careful to support large numbers

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
from styles import CSS

with gr.Blocks(css=CSS, theme=gr.themes.Monochrome(), title=f"Format Python to to be Ruff code quality ") as ui:
    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python = gr.Code(
                label="Python (original)",
                value=python_code,
                language="python",
                lines=26
            )
        with gr.Column(scale=6):
            ruff_compliant = gr.Code(
                label=f"Ruff compliant Python (generated)",
                value="",
                language="python",
                lines=26
            )

    with gr.Row(elem_classes=["controls"]):
        python_run = gr.Button("Run Python Code", elem_classes=["run-btn", "py"])
        model = gr.Dropdown(models, value=models[0], show_label=False)
        convert = gr.Button(f"Format code (Ruff)", elem_classes=["convert-btn"])
        formatted_code_run = gr.Button(f"Run formatted code", elem_classes=["run-btn", "cpp"])

    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python_out = gr.TextArea(label="Python result", lines=8, elem_classes=["py-out"])
        with gr.Column(scale=6):
            formatted_code_out = gr.TextArea(label=f"Formatted code result", lines=8, elem_classes=["formatted-code-out"])

    convert.click(fn=implement_ruff_code_quality, inputs=[model, python], outputs=[ruff_compliant])
    python_run.click(fn=run_code, inputs=[python], outputs=[python_out])
    formatted_code_run.click(fn=run_code, inputs=[ruff_compliant], outputs=[formatted_code_out])

ui.launch(inbrowser=True)
